In [1]:
import sqlite3
import pandas as pd

CATALOG_DB_PATH = "/Users/seunfuta/Downloads/NIST/OluDB_combo_v3.db"
catalog_conn = sqlite3.connect(CATALOG_DB_PATH)
catalog_df = pd.DataFrame(columns=['obj_id', 'inode', 'filename','file_offset', 'len','md5','sha1', 'partition', 'filesize','app','app_id'])
catalog_df = pd.read_sql_query("SELECT block_hashes.obj_id, block_hashes.inode, block_hashes.filename, block_hashes.file_offset, \
                block_hashes.len, block_hashes.md5, block_hashes.sha1, files.partition,files.filesize , files.app, files.app_id\
                FROM files \
                INNER JOIN block_hashes ON files.obj_id = block_hashes.obj_id \
                and files.inode = block_hashes.inode and files.filename=block_hashes.filename;", catalog_conn)
print("original length ",len(catalog_df))
catalog_df = catalog_df[catalog_df.md5 != 'bf619eac0cdf3f68d496ea9344137e8b']
catalog_df = catalog_df[catalog_df.md5 != 'de03fe65a6765caa8c91343acc62cffc']
catalog_df = catalog_df[catalog_df.md5 != '85eba416ce0ee0951d1d93e73b191b75']
catalog_df = catalog_df[catalog_df.md5 != '1b5c2cbf1e37f6b0d33751269ae707af']
catalog_conn.close()
print("catalog app length, ", len(catalog_df))
#return appdf

original length  6826014
catalog app length,  6546710


In [2]:
IMAGE_DB_PATH = "/Volumes/Samsung_T5/M57/pat2.db"
image_conn = sqlite3.connect(IMAGE_DB_PATH)
image_df = pd.DataFrame(columns=['obj_id', 'inode', 'filename','file_offset', 'len','md5','sha1', 'partition', 'filesize'])
image_df = pd.read_sql_query("SELECT block_hashes.obj_id, files.inode, files.filename, block_hashes.file_offset, \
                block_hashes.len, block_hashes.md5, block_hashes.sha1, files.partition,files.filesize \
                FROM files \
                INNER JOIN block_hashes ON files.obj_id = block_hashes.obj_id;", image_conn)
print("original image length ",len(image_df))
image_df = image_df[image_df.md5 != 'bf619eac0cdf3f68d496ea9344137e8b']
image_df = image_df[image_df.md5 != 'de03fe65a6765caa8c91343acc62cffc']
image_df = image_df[image_df.md5 != '85eba416ce0ee0951d1d93e73b191b75']
image_df = image_df[image_df.md5 != '1b5c2cbf1e37f6b0d33751269ae707af']
print("current image length ",len(image_df))

original image length  15413093
current image length  14703514


In [3]:
import numpy as np
app_list = catalog_df.app.unique()
result_df = pd.DataFrame()
for app in app_list:
    #if app == 'OfficePro2003-W7x32':
    app_df = catalog_df[catalog_df.app == app]
    app_unique_md5s= app_df.md5.unique()
    matched_image_df = image_df[image_df.md5.isin(app_unique_md5s)]
    #print(app," matched number of sectors in image ",len(matched_image_df))
    #print(matched_image_df)
    #lets create app sec pairs
    app_files = app_df.filename.unique()
    #print(app_files)
    app_pairs_set = {} #dict #set() 
    for file in app_files:
        file_hashpair_set = set()
        files_df = app_df[app_df.filename == file]
        #print(f'file {file} is of size {len(files_df)}')
        file_hashes = files_df.md5
        if len(file_hashes)< 2:
            #print(file_hashes.iloc[0])
            file_hashpair_set.add(file_hashes.iloc[0])
        else:
            for i in range(0, len(file_hashes)-1):
                #print(file_hashes.iloc[i])
                #print(file_hashes.iloc[int(i+1)])
                hash_pair = file_hashes.iloc[i]+file_hashes.iloc[i+1]
            #print(i, hash_pair)
                file_hashpair_set.add(hash_pair)
        app_pairs_set[file] = file_hashpair_set 
    #print(f"set size is {len(app_pairs_set)}")
    lst2 = list(matched_image_df.md5)
    lst2_pairs = list(map(lambda a, b: a + b, lst2[:-1], lst2[1:]))
    Prob_Total = 0
    for file in app_files:
        #x = 0 #setting a default value
        forward_list = list(map(lambda x: 1 if x in app_pairs_set[file] else 0, lst2_pairs))
        x= forward_list.count(1)
        t= len(forward_list)+0.0000000001
        s = 1
        q = 2
        Prob_file = (1 - ((1/(x + s))**q))**(np.log(t))
        Prob_Total+= Prob_file
    Prob_App = Prob_Total/len(app_files)    


    #forward_series = pd.Series(forward_list)
    if Prob_App == np.inf: Prob_App =float(0)
    print(app, " matched ", len(matched_image_df), " set ", len(app_pairs_set), " Prob ",Prob_App)
    result_df.loc[app,'matched'] = len(matched_image_df)
    result_df.loc[app,'prob'] = "{:.4f}".format(Prob_App)
print(result_df)

Wireshark-W7x64  matched  5239  set  668  Prob  0.004153327669608866
Wireshark-W7x32  matched  5893  set  671  Prob  0.007217902258553911
Winzip17pro-W7x32  matched  22010  set  172  Prob  0.2499943206842112
Winzip17pro-W7x64  matched  21698  set  175  Prob  0.21929526947629407
sdelete-W7x32  matched  31  set  5  Prob  0.38655778726414197
sdelete-W7x64  matched  31  set  3  Prob  0.64426297877357
OfficePro2003-WinXP  matched  21115  set  3348  Prob  0.010939224878364143
OfficePro2003-W7x64  matched  19469  set  3235  Prob  0.009969058434300264
OfficePro2003-W7x32  matched  20576  set  3233  Prob  0.010742690690693455
Winrar5beta-W7x32  matched  360  set  47  Prob  0.165369568008191
Firefox19-WinXP  matched  1475  set  109  Prob  0.06469444399475519
Winrar5beta-W7x64  matched  360  set  47  Prob  0.165369568008191
Firefox19-W7x32  matched  1475  set  109  Prob  0.06469444399475519
HxD171-W7x32  matched  1374  set  15  Prob  0.0
Firefox19-W7x64  matched  1484  set  207  Prob  0.034060056

/var/folders/s7/d__51l693s13d_yt81j0qp9r0000gn/T/ipykernel_41384/279786015.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  Prob_file = (1 - ((1/(x + s))**q))**(np.log(t))


UPX-W7x32  matched  0  set  14  Prob  0.0
UPX-W7x64  matched  0  set  14  Prob  0.0
                          matched    prob
Wireshark-W7x64            5239.0  0.0042
Wireshark-W7x32            5893.0  0.0072
Winzip17pro-W7x32         22010.0  0.2500
Winzip17pro-W7x64         21698.0  0.2193
sdelete-W7x32                31.0  0.3866
sdelete-W7x64                31.0  0.6443
OfficePro2003-WinXP       21115.0  0.0109
OfficePro2003-W7x64       19469.0  0.0100
OfficePro2003-W7x32       20576.0  0.0107
Winrar5beta-W7x32           360.0  0.1654
Firefox19-WinXP            1475.0  0.0647
Winrar5beta-W7x64           360.0  0.1654
Firefox19-W7x32            1475.0  0.0647
HxD171-W7x32               1374.0  0.0000
Firefox19-W7x64            1484.0  0.0341
Thunderbird2-WinXP         1101.0  0.0311
Python264-WinXP           91931.0  0.0701
eraser-W7x32                288.0  0.0802
Chrome28-W7x64             9627.0  0.0181
Chrome28-WinXP            10210.0  0.0192
Chrome28-W7x32             9625.0 

/var/folders/s7/d__51l693s13d_yt81j0qp9r0000gn/T/ipykernel_41384/279786015.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  Prob_file = (1 - ((1/(x + s))**q))**(np.log(t))


In [5]:
print(result_df.head(5))

                    matched      prob
Wireshark-W7x64     56416.0  0.726628
Wireshark-W7x32    171330.0  0.774965
Winzip17pro-W7x32     184.0  0.120595
Winzip17pro-W7x64      95.0  0.107737
sdelete-W7x32           8.0  0.000000


In [6]:
result_df.to_csv("/Users/seunfuta/Downloads/test.csv")

In [ ]:
app_list = catalog_df.app.unique()
for app in app_list:
    if app == 'Firefox19-W7x64':
        app_df = catalog_df[catalog_df.app == app]
        app_unique_md5s= app_df.md5.unique()
        matched_image_df = image_df[image_df.md5.isin(app_unique_md5s)]
        print(app," matched number of sectors in image ",len(matched_image_df))
        #print(matched_image_df)
        #lets create app sec pairs
        app_files = app_df.filename.unique()
        #print(app_files)
        app_dict_set = {}
        app_pairs_set = set() 
        for file in app_files:
            files_df = app_df[app_df.filename == file]
            #print(f'file {file} is of size {len(files_df)}')
            file_hashes = files_df.md5
            if len(file_hashes)< 2:
                #print(file_hashes.iloc[0])
                #app_pairs_set.add(file_hashes.iloc[0])
                app_dict_set[file]=set(file_hashes.iloc[0])
            else:
                app_dict_set[file] = set()
                for i in range(0, len(file_hashes)-1):
                    #print(file_hashes.iloc[i])
                    #print(file_hashes.iloc[int(i+1)])
                    hash_pair = file_hashes.iloc[i]+file_hashes.iloc[i+1]
                    #print(i, hash_pair)
                    #app_pairs_set.add(hash_pair)
                    app_dict_set[file].add(hash_pair)
        #print(f"set size is {len(app_pairs_set)}")
        #for key,value in app_dict_set.items():
            #print(key,len(value))
        image_md5_list = list(matched_image_df.md5)
        image_md5_pairs = list(map(lambda a, b: a + b, image_md5_list[:-1], image_md5_list[1:]))
        for key,value in app_dict_set.items():
            forward_list = list(map(lambda x: 1 if x in value else 0, image_md5_pairs))
            #forward_x = list(map(lambda x: x if x in value else 0, image_md5_pairs))
            forward_series = pd.Series(forward_list).value_counts()
            if 1 in forward_series.index.values:
                print(key,forward_series[1])#,"\n",forward_x)
        #print(app, "matched", len(matched_image_df), "set", len(app_pairs_set), "valid",forward_series.value_counts())

In [ ]:
app_list = catalog_df.app.unique()
for app in app_list:
    if app == 'Firefox19-W7x64':
        app_df = catalog_df[catalog_df.app == app]
        app_unique_md5s= app_df.md5.unique()
        matched_image_df = image_df[image_df.md5.isin(app_unique_md5s)]
        print(app," matched number of sectors in image ",len(matched_image_df))
        #print(matched_image_df)
        #lets create app sec pairs
        app_files = app_df.filename.unique()
        #print(app_files)
        app_dict_set = {}
        app_pairs_set = set() 
        for file in app_files:
            files_df = app_df[app_df.filename == file]
            #print(f'file {file} is of size {len(files_df)}')
            file_hashes = files_df.md5
            if len(file_hashes)< 2:
                #print(file_hashes.iloc[0])
                #app_pairs_set.add(file_hashes.iloc[0])
                app_dict_set[file]=set(file_hashes.iloc[0])
            else:
                app_dict_set[file] = set()
                for i in range(0, len(file_hashes)-1):
                    #print(file_hashes.iloc[i])
                    #print(file_hashes.iloc[int(i+1)])
                    hash_pair = file_hashes.iloc[i]+file_hashes.iloc[i+1]
                    #print(i, hash_pair)
                    #app_pairs_set.add(hash_pair)
                    app_dict_set[file].add(hash_pair)
        #print(f"set size is {len(app_pairs_set)}")
        #for key,value in app_dict_set.items():
            #print(key,len(value))
        image_md5_list = list(matched_image_df.md5)
        image_md5_pairs = list(map(lambda a, b: a + b, image_md5_list[:-1], image_md5_list[1:]))
        for key,value in app_dict_set.items():
            forward_list = list(map(lambda x: 1 if x in value else 0, image_md5_pairs))
            forward_x = list(map(lambda x: x if x in value else 0, image_md5_pairs))
            forward_series = pd.Series(forward_list).value_counts()
            if 1 in forward_series.index.values:
                print(key,forward_series[1],"\n",forward_x)
        #print(app, "matched", len(matched_image_df), "set", len(app_pairs_set), "valid",forward_series.value_counts())

In [ ]:
import sqlite3
import pandas as pd

CATALOG_DB_PATH = "/Users/seunfuta/Downloads/NIST/OluDB_combo_v3.db"
catalog_conn = sqlite3.connect(CATALOG_DB_PATH)
catalog_df = pd.DataFrame(columns=['obj_id', 'inode', 'filename','file_offset', 'len','md5','sha1', 'partition', 'filesize','app','app_id'])
catalog_df = pd.read_sql_query("SELECT block_hashes.obj_id, block_hashes.inode, block_hashes.filename, block_hashes.file_offset, \
                block_hashes.len, block_hashes.md5, block_hashes.sha1, files.partition,files.filesize , files.app, files.app_id\
                FROM files \
                INNER JOIN block_hashes ON files.obj_id = block_hashes.obj_id \
                and files.inode = block_hashes.inode and files.filename=block_hashes.filename;", catalog_conn)
print("original length ",len(catalog_df))
catalog_df = catalog_df[catalog_df.md5 != 'bf619eac0cdf3f68d496ea9344137e8b']
catalog_df = catalog_df[catalog_df.md5 != 'de03fe65a6765caa8c91343acc62cffc']
catalog_df = catalog_df[catalog_df.md5 != '85eba416ce0ee0951d1d93e73b191b75']
catalog_df = catalog_df[catalog_df.md5 != '1b5c2cbf1e37f6b0d33751269ae707af']
catalog_conn.close()
print("catalog app length, ", len(catalog_df))
print(catalog_df.head(5))

In [ ]:
print(len(catalog_df[catalog_df.app == 'TrueCrypt63-WinXP']))

In [ ]:
appx_df = catalog_df[catalog_df.app == 'TrueCrypt63-WinXP']